In [ ]:
#!/usr/bin/env python

# Copyright 2018 Paul Archer
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import time
import json
from datetime import datetime
from sys import stdout
import argparse

from pvstats.pvinverter.factory import PVInverterFactory
from pvstats.report import PVReportFactory

import logging
import traceback

# Setup the logging
logging.basicConfig()
_log = logging.getLogger()


# Load the configuration
def load_config(location):
  with open(location) as f:
    cfg = json.load(f)

  try:
    if cfg.get('log_level') is not None:
      _log.setLevel(cfg['log_level'])
  except:
    _log.setLevel('INFO')
  return cfg

def main(arg=None):
  # Parse input arguments
  if not arg:
      parser = argparse.ArgumentParser(
          description="Photovoltaic Inverter Statistics Scanner and Uploader",
          prog="pvstats",
          usage="%(prog)s -[options]")
      parser.add_argument("--cfg", help="Configuration File", nargs=1, default="/etc/pvstats.conf")
      args = parser.parse_args()

  # Initialise
      cfg = load_config(vars(args)['cfg'][0])
  else:
    cfg = load_config(arg)
    

  # Get an PV inverter client
  inverter = PVInverterFactory(cfg['inverter']['model'], cfg['inverter'])

  # Create the report channels
  reports = []
  for rpt in cfg['reports']:
    _log.debug(json.dumps(rpt, sort_keys=True,
                         indent=4, separators=(',', ': '),default=str))
    r = PVReportFactory(rpt)
    if r != None:
      reports.append(r)

  while True:
    tstart = datetime.now()
    try:
      # Grab the data from the inverter
      inverter.connect()
      inverter.read()

      # Log it
      _log.debug(json.dumps(inverter.registers, sort_keys=True,
                           indent=4, separators=(',', ': '),default=str))

      # Publish it
      for rpt in reports:
        rpt.publish(inverter.registers)

    except Exception as err:
      _log.debug(traceback.format_exc())
      _log.debug("Ignoring = {}".format(err))

    finally:
      inverter.close()

    tend = datetime.now()
    time.sleep(max((tstart - tend).total_seconds() + cfg['sample_period'],0))


if __name__ == "__main__":
  main("..\pvstats.conf")

# vim: set expandtab ts=2 sw=2:


INFO:root:[INFO] Sent to InfluxDB
INFO:root:[INFO] Sent to InfluxDB
